# Cylindrical compressive test

In [ ]:
%matplotlib widget
from ibvpy.bcond import BCSlice
from ibvpy.fets import FETS2D4Q
from ibvpy.tmodel.mats3D import MATS3DElastic
from bmcs_matmod.ms1.ms1 import MS13D
import bmcs_matmod.ms1.concrete_material_db as mp_db
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField
from mayavi import mlab
from ibvpy.sim.tstep_bc import TStepBC
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym

import numpy as np


In [ ]:
from bmcs_expsim.utils.mlab_decorators import decorate_figure

# a = pi r**2
# r = np.sqrt( a / pi )

unit_radius = np.sqrt( 1 / np.pi )
unit_height = 1
unit_radius

In [ ]:
inner_radius = 0
outer_radius = unit_radius
height = unit_height

xmodel = XDomainFEGridAxiSym(coord_min=(0,inner_radius),
                             coord_max=(height,outer_radius),
                             shape=(2, 2),
                             integ_factor=2 * np.pi,
                             fets=FETS2D4Q())

In [ ]:
tmodel = MS13D(**mp_db.C40MS1)
#tmodel = MATS3DElastic(E=1, nu=0)

w_max = -0.01
bc_ax_sym = BCSlice(slice=xmodel.mesh[:, 0, :, 0],
                 var='u', dims=[1], value=0)
bc_l_x = BCSlice(slice=xmodel.mesh[0, :, 0, :],
                  var='u', dims=[0], value=0.0)
bc_r_x = BCSlice(slice=xmodel.mesh[-1, :, -1, :],
                 var='u', dims=[0], value=w_max)
# bc_r_y = BCSlice(slice=xmodel.mesh[-1, :, -1, :],
#                   var='u', dims=[1], value=0)

In [ ]:
m = TStepBC(
    domains=[(xmodel, tmodel)],
    bc=[bc_ax_sym, bc_l_x, bc_r_x]
)

m.hist.vis_record = {
    'strain': Vis3DTensorField(var='eps_ab'),
    # 'stress': Vis3DTensorField(var='sig_ab'),
    #        'damage': Vis3DStateField(var='omega_a'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

In [ ]:
w_steps = 200
s = m.sim
s.tloop.k_max = 1000
s.tline.step = 1 / w_steps
s.tloop.verbose = True
s.run()

In [ ]:
F_to = m.hist.F_t
U_to = m.hist.U_t

In [ ]:
F_l_t = np.sum(F_to[:, bc_l_x.dofs], axis=-1)
F_r_t = np.sum(F_to[:, bc_r_x.dofs], axis=-1)
U_r_t = np.average(U_to[:, bc_r_x.dofs], axis=-1)

import matplotlib.pylab as plt
plt.plot(-U_r_t, -F_r_t)
plt.plot(-U_r_t, F_l_t)

In [ ]:
F_l_t[-1]